In [0]:
DEBUG = False
CONFIG_FILE = '/datascope/subaru/data/targeting/MW/outerdisk_l180_b22_ENG/netflow/outerdisk_l180_b22_faint_004/ga-netflow_20250123033847.cmd'
OUTPUT_PATH = '/datascope/subaru/data/targeting/MW/outerdisk_l180_b22_ENG/netflow/outerdisk_l180_b22_faint_004/'

# Plot the design files

In [0]:
import os, sys, re
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [0]:
plt.rc('font', size=6) #controls default text size

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
if DEBUG and 'debug' not in globals():
    import debugpy
    debugpy.listen(('0.0.0.0', int(os.environ['PFS_TARGETING_DEBUGPORT'])))
    debug = True

# Imports

In [0]:
import pfs.utils
from pfs.datamodel import PfsDesign, FiberStatus, TargetType

from pfs.ga.targeting.scripts.netflow.notebooks.notebooks import *
from pfs.ga.targeting.instrument import *
from pfs.ga.targeting.util.astro import *

# Load the netflow config file

In [0]:
# Load the configuration
config = load_netflow_config(CONFIG_FILE)

# Plot the design files

In [0]:
# Create the PFI instrument. This will load the instrument calibration data.
pfi = SubaruPFI(instrument_options=config.instrument_options)

In [0]:
# Find and load the design files in the output directory
designs = {}
design_files = glob(os.path.join(OUTPUT_PATH, 'pfsDesign*.fits'))
for fn in design_files:
    pfsDesignId = int(re.search(r'0x[0-9a-f]+', fn).group(0), 16)
    design = PfsDesign.read(pfsDesignId, dirName=os.path.dirname(fn))
    designs[pfsDesignId] = design
    print(f'Loaded design 0x{pfsDesignId:016x}')

In [0]:
# Verify that there are no duplicate fiberIds
for pfsDesignId, design in designs.items():
    counts = np.bincount(design.fiberId)
    print(f'0x{pfsDesignId:016x}', np.where(counts > 1))

In [0]:
for pfsDesignId, design in designs.items():
        f, ax = plt.subplots(1, 1, figsize=(4, 4), dpi=240)

        # Broken cobras or fibers
        mask = (design.fiberStatus != FiberStatus.GOOD)
        ax.plot(design.pfiNominal[mask][..., 0], design.pfiNominal[mask][..., 1], 'x', ms=2, label='broken',
                color='black', markeredgewidth=0.5)

        # Unassigned good cobras
        mask = (design.fiberStatus == FiberStatus.GOOD) & (design.targetType == TargetType.UNASSIGNED)
        ax.plot(design.pfiNominal[mask][..., 0], design.pfiNominal[mask][..., 1], 'o', ms=2, label='unassigned',
                markerfacecolor='none', markeredgecolor='black', markeredgewidth=0.2)

        # Good science targets
        mask = (design.fiberStatus == FiberStatus.GOOD) & (design.targetType == TargetType.SCIENCE)
        ax.plot(design.pfiNominal[mask][..., 0], design.pfiNominal[mask][..., 1], 'o', ms=1, label='science', color='green')

        # Good sky fibers
        mask = (design.fiberStatus == FiberStatus.GOOD) & (design.targetType == TargetType.SKY)
        ax.plot(design.pfiNominal[mask][..., 0], design.pfiNominal[mask][..., 1], 'o', ms=1, label='sky', color='lightblue')

        # Good flux standards
        mask = (design.fiberStatus == FiberStatus.GOOD) & (design.targetType == TargetType.FLUXSTD)
        ax.plot(design.pfiNominal[mask][..., 0], design.pfiNominal[mask][..., 1], 'o', ms=1, label='fluxstd', color='red')

        # Cobra centers
        ax.scatter(pfi.bench.cobras.centers.real, pfi.bench.cobras.centers.imag, 
                   s=0.2, color='black', edgecolors='none')

        ax.set_title(
                f'designId=0x{pfsDesignId:016x} ({design.raBoresight:0.02f}, {design.decBoresight:0.02f}, PA={design.posAng:0.01f})\n'
                f'{design.designName}')

        # Make legend horizontal
        ax.legend(loc='upper right', ncol=2, fontsize='small')

        f.tight_layout()

In [0]:
cmap = plt.get_cmap('tab10')

for pfsDesignId, design in designs.items():
    f, ax = plt.subplots(1, 1, figsize=(3.5, 2), dpi=240)
    for j, target_type in enumerate([TargetType.SCIENCE, TargetType.SKY, TargetType.FLUXSTD]):
        idx = np.where((design.fiberStatus == FiberStatus.GOOD) & (design.targetType == target_type))[0]
        flux = np.array([ design.fiberFlux[i][0] for i in idx if len(design.fiberFlux[i]) > 0 ])
        mag = nJy_to_ABmag(flux)
        
        ax.hist(mag, color=cmap(j), label=target_type)
        
    ax.set_xlim(13.1, 24.9)

In [0]:
# Unique catIds
for pfsDesignId, design in designs.items():
    print(f'0x{pfsDesignId:016x}', np.unique(design.catId))

In [0]:
# Unique proposalIds
for pfsDesignId, design in designs.items():
    print(f'0x{pfsDesignId:016x}', np.unique(design.catId))

In [0]:
# objId ranges
for pfsDesignId, design in designs.items():
    for target_type in [TargetType.SCIENCE, TargetType.SKY, TargetType.FLUXSTD]:
        mask = (design.fiberStatus == FiberStatus.GOOD) & (design.targetType == target_type)
        if mask.sum() > 0:
            print(f'0x{pfsDesignId:016x}', target_type, design.objId[mask].min(), design.objId[mask].max())
        else:
            print(f'0x{pfsDesignId:016x}', target_type, 'no such targets in design')